In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
# import other packages

import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')

from pandas_profiling import ProfileReport
from IPython.display import HTML

from tqdm import tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

%matplotlib inline
plt.rcParams["figure.figsize"] = (16,12)
plt.rcParams['axes.titlesize'] = 16


In [21]:
# Read the input data

train_sales = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
submission_file = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')

In [22]:
#EDA - Training sales dataset

train_sales.head()


In [23]:
train_sales.shape

In [24]:
#Helper function for box plots

def disp_boxplot(data, title, xlabel, ylabel):
    sns.set_style('whitegrid')
    sns.set_context('poster')
    palette = sns.color_palette("mako_r", 8)
    
    ax = sns.boxplot(data = data, palette = palette)
    ax.set(title = title,
          xlabel = xlabel,
          ylabel = ylabel)
    
    try:
        ax.ahline(y = data.mean().mean(), color = 'b',label = 'Mean', linestyle = '--', linewidth = 1.5)
        ax.ahline(y = data.median().median(), color = 'g',label = 'Median', linestyle = '--', linewidth = 1.5)
    except:
        pass
    
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45)
    
    plt.legend()
    plt.show()

In [25]:
# Sales by dept

dept_sales = train_sales.groupby(['dept_id']).mean().mean()
dept_sum = train_sales.groupby(['dept_id']).sum().T.reset_index(drop = True)

In [26]:
disp_boxplot(data = dept_sum, title = 'Sales by dept', xlabel = 'Department', ylabel = 'Total sales')

In [27]:
# Sales by store

store_sales = train_sales.groupby(['store_id']).mean().mean()
store_sum = train_sales.groupby(['store_id']).sum().T.reset_index(drop = True)

disp_boxplot(data = store_sum, title = 'Sales by store', xlabel = 'Store', ylabel = 'Total sales')

In [28]:
# Sales by category

cat_sales = train_sales.groupby(['cat_id']).mean().mean()
cat_sum = train_sales.groupby(['cat_id']).sum().T.reset_index(drop = True)

disp_boxplot(data = cat_sum, title = 'Sales by category', xlabel = 'Category', ylabel = 'Total sales')

In [29]:
# Sales by state

state_sales = train_sales.groupby(['state_id']).mean().mean()
state_sum = train_sales.groupby(['state_id']).sum().T.reset_index(drop = True)

disp_boxplot(data = state_sum, title = 'Sales by state', xlabel = 'State', ylabel = 'Total sales')

In [30]:
# Merchandise distribution by store

merch = pd.crosstab(train_sales['dept_id'], train_sales['store_id'])
print(merch)


In [31]:
# Wavelet_denoising. 
# Reference : 
#Code Source : https://www.kaggle.com/tarunpaparaju/m5-competition-eda-models

def mad(d,axis = None):
    return np.mean(np.absolute(d - np.mean(d,axis)), axis)

import pywt
def denoise_signal(x, wavelet = 'db4', level = 1):
    coeff = pywt.wavedec(x,wavelet, mode = "per")
    sigma = (1/0.6745) * mad(coeff[-level])
    
    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i,value = uthresh, mode = 'hard') for i in coeff[1:])
    
    return pywt.waverec(coeff, wavelet, mode = 'per')







In [40]:
# Splitting sales by state

CA_sales = state_sum['CA']
TX_sales = state_sum['TX']
WI_sales = state_sum['WI']



In [42]:
total_sales = CA_sales + TX_sales + WI_sales
total_sales.head()

In [41]:
y_w1 = denoise_signal(CA_sales)
y_w2 = denoise_signal(TX_sales)
y_w3 = denoise_signal(WI_sales)


fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(len(CA_sales)), mode='lines+markers', y=CA_sales, marker=dict(color="mediumaquamarine"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(CA_sales)), y=y_w1, mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(TX_sales)), mode='lines+markers', y=TX_sales, marker=dict(color="thistle"), showlegend=False,
               name="Original signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(TX_sales)), y=y_w2, mode='lines', marker=dict(color="indigo"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(WI_sales)), mode='lines+markers', y=WI_sales, marker=dict(color="lightskyblue"), showlegend=False,
               name="Original signal"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(WI_sales)), y=y_w3, mode='lines', marker=dict(color="navy"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)




fig.update_layout(height=1200, width=800, title_text="Original (pale) vs. Denoised (dark) sales")
fig.show()

In [44]:
#Sales decomposition

from statsmodels.tsa.seasonal import seasonal_decompose
days_per_yr = 365

sd = seasonal_decompose(total_sales[1:,], period = days_per_yr)

sd.plot()




In [47]:
# Split data for Modelling 

d_cols = [c for c in train_sales.columns if 'd_' in c]

train_data = train_sales[d_cols[-100:-25]]
val_data = train_sales[d_cols[-25:]]



In [49]:
# Naive approach 

predictions = []
for i in range(len(val_data.columns)):
    if i == 0:
        predictions.append(train_data[train_data.columns[-1]].values)
    else:
        predictions.append(val_data[val_data.columns[i-1]].values)
        

predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_naive = np.linalg.norm(predictions[:3] - val_data.values[:3])/len(predictions[0])

In [50]:
predictions[0]

In [51]:
pred_1 = predictions[0]
pred_2 = predictions[1]
pred_3 = predictions[2]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"),
               name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[0].values, mode='lines', marker=dict(color="darkorange"),
               name="Val"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Naive approach")
fig.show()

In [54]:
# Moving average

predictions = []
for i in range(len(val_data.columns)):
    if i == 0:
        predictions.append(np.mean(train_data[train_data.columns[-30:]].values, axis=1))
    if i < 31 and i > 0:
        predictions.append(0.5 * (np.mean(train_data[train_data.columns[-30+i:]].values, axis=1) + \
                                  np.mean(predictions[:i], axis=0)))
    if i > 31:
        predictions.append(np.mean([predictions[:i]], axis=1))
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_avg = np.linalg.norm(predictions[:3] - val_data.values[:3])/len(predictions[0])

predictions = []
for i in range(len(val_data.columns)):
    if i == 0:
        predictions.append(np.mean(train_data[train_data.columns[-30:]].values, axis=1))
    if i < 31 and i > 0:
        predictions.append(0.5 * (np.mean(train_data[train_data.columns[-30+i:]].values, axis=1) + \
                                  np.mean(predictions[:i], axis=0)))
    if i > 31:
        predictions.append(np.mean([predictions[:i]], axis=1))
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_avg = np.linalg.norm(predictions[:3] - val_data.values[:3])/len(predictions[0])

In [55]:
pred_1 = predictions[0]
pred_2 = predictions[1]
pred_3 = predictions[2]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"),
               name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[0].values, mode='lines', marker=dict(color="darkorange"),
               name="Val"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Moving average")
fig.show()

In [57]:
# Holt Winters model

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt


In [63]:

predictions = []
for row in tqdm(train_data[train_data.columns[-25:]].values[:3]):
    fit = Holt(row).fit(smoothing_level = 0.3, smoothing_slope = 0.01)
    predictions.append(fit.forecast(25))
predictions = np.array(predictions).reshape((-1, 25))
error_holt = np.linalg.norm(predictions - val_data.values[:len(predictions)])/len(predictions[0])

In [65]:
pred_1 = predictions[0]
pred_2 = predictions[1]
pred_3 = predictions[2]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"),
               name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[0].values, mode='lines', marker=dict(color="darkorange"),
               name="Val"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_data.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Holt linear")
fig.show()